### This code will run only in google colab

In [1]:
%pip install -Uq "unstructured[all-docs]"
%pip install -Uq langchain_chroma
%pip install -Uq langchain langchain-community langchain-openai
%pip install -Uq python_dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.9/527.9 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 118.1 MB/s eta 0:00:00


In [2]:
import json
from typing import List

from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title

from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.messages import HumanMessage

In [3]:
def partition_document(file_path:str):
    print(f"partition document: {file_path}")

    elements = partition_pdf(
        filename = file_path,
        strategy="hi_res",
        infer_table_structure=True,
        extract_image_block_types=["Image"],
        extract_image_block_to_payload=True
    )

    print(f"Extracted {len(elements)} elements")
    return elements

In [4]:
!sudo apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 0s (608 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 126666

In [5]:
file_path="/content/3723178.3723198.pdf"
elements = partition_document(file_path=file_path)

partition document: /content/3723178.3723198.pdf


yolox_l0.05.onnx:   0%|          | 0.00/217M [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Extracted 195 elements


In [6]:
elements

In [7]:
set([str(type(el)) for el in elements])


{"<class 'unstructured.documents.elements.FigureCaption'>",
 "<class 'unstructured.documents.elements.Footer'>",
 "<class 'unstructured.documents.elements.Header'>",
 "<class 'unstructured.documents.elements.Image'>",
 "<class 'unstructured.documents.elements.ListItem'>",
 "<class 'unstructured.documents.elements.NarrativeText'>",
 "<class 'unstructured.documents.elements.Table'>",
 "<class 'unstructured.documents.elements.Text'>",
 "<class 'unstructured.documents.elements.Title'>"}

In [8]:
elements[2].to_dict()

{'type': 'NarrativeText',
 'element_id': '97c93816158eddc5f5f562c74c203a6c',
 'text': 'Al-Momen Reyad Computer Science and Engineering United International University Dhaka, Bangladesh areyad203011@bscse.uiu.ac.bd',
 'metadata': {'detection_class_prob': 0.8866927027702332,
  'coordinates': {'points': ((np.float64(195.79999999999998),
     np.float64(353.9244444444443)),
    (np.float64(195.79999999999998), np.float64(525.2470703125)),
    (np.float64(595.0235211111112), np.float64(525.2470703125)),
    (np.float64(595.0235211111112), np.float64(353.9244444444443))),
   'system': 'PixelSpace',
   'layout_width': 1700,
   'layout_height': 2200},
  'links': [{'text': 'Al - Momen Reyad',
    'url': 'https://orcid.org/0009-0005-3905-6073',
    'start_index': 0},
   {'text': 'Al - Momen Reyad',
    'url': 'https://orcid.org/0009-0005-3905-6073',
    'start_index': 0}],
  'last_modified': '2025-10-03T08:34:34',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'fi

In [9]:
images = [element for element in elements if element.category == 'Image']
print(f"Found {len(images)} images")

Found 12 images


In [10]:
images[1].to_dict()


{'type': 'Image',
 'element_id': '706dfc2e7216cbae934e75b86fc8cb3b',
 'text': 'Train(80%) Coccidiosisd Salmonellosis + Rotation wealthy + Tooming New- castle disease Data Augmentation ightmess adjustment + Slight image distortion Dataset Test(20%) Classes Data preprocessing Prediction Model Train Majority voting',
 'metadata': {'coordinates': {'points': ((np.float64(149.4388888888889),
     np.float64(978.5530902777778)),
    (np.float64(149.4388888888889), np.float64(1452.5777777777778)),
    (np.float64(1550.5450347222222), np.float64(1452.5777777777778)),
    (np.float64(1550.5450347222222), np.float64(978.5530902777778))),
   'system': 'PixelSpace',
   'layout_width': 1700,
   'layout_height': 2200},
  'last_modified': '2025-10-03T08:34:34',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 3,
  'image_base64': '...',
  'image_mime_type': 'image/jpeg',
  'file_directory': '/content',
  'filename': '3723178.3723198.pdf'}}

In [11]:
tables = [element for element in elements if element.category == 'Table']
print(f"Found {len(tables)} tables")

tables[0].to_dict()

Found 4 tables


{'type': 'Table',
 'element_id': '14ae72dd57677b4a5ad14c2443a4c168',
 'text': 'References Contribution Limitation Algorithm Accuracy Hossain et Smart poultry system app with Overlooks practical implementation Ensemble learning with 99.99% al.[2] exceptional accuracy using en- challenges like cost and continuous up- Random Forest semble learning dates Degu et Developed a smartphone-based Does not address real world challenges YOLO-V3 for ROI seg- 87.48% al.[9] system for classification of poul- like lighting conditions mentation, ResNet50 and 98.7% try diseases from fecal images for classification Suthagar et Early detection and classifica- Small dataset size and lack of explo- DenseNet, Inception, 97%, 92% al.[10] tion of poultry diseases using ration of varying environmental con- MobileNet and 94% CNNs ditions Chen et Improved ResNeXt50 model for Model performance and generalizability ResNeXt50-3A with 97.4% al.[11] disease recognition in broiler not assessed across diverse conditions

In [12]:
def create_chunks_by_title(elements):
  chunks = chunk_by_title(
      elements=elements,
      max_characters=3000,
      new_after_n_chars=2400,
      combine_text_under_n_chars=500
  )

  print(f"Created {len(chunks)} chunks")
  return chunks

chunks = create_chunks_by_title(elements)

Created 21 chunks


In [13]:
chunks

In [14]:
set([str(type(chunk)) for chunk in chunks])


{"<class 'unstructured.documents.elements.CompositeElement'>"}

In [15]:
chunks[3].to_dict()

{'type': 'CompositeElement',
 'element_id': '790801d4-745b-4c04-896f-544fc480b34c',
 'text': 'CCS Concepts\n\n• Computing methodologies → Computer vision.\n\nPermission to make digital or hard copies of all or part of this work for personal or classroom use is granted without fee provided that copies are not made or distributed for profit or commercial advantage and that copies bear this notice and the full citation on the first page. Copyrights for components of this work owned by others than the author(s) must be honored. Abstracting with credit is permitted. To copy otherwise, or republish, to post on servers or to redistribute to lists, requires prior specific permission and/or a fee. Request permissions from permissions@acm.org.\n\nICCA 2024, Dhaka, Bangladesh\n\n© 2024 Copyright held by the owner/author(s). Publication rights licensed to ACM. ACM ISBN 979-8-4007-1382-8/24/10\n\nhttps://doi.org/10.1145/3723178.3723198\n\nNew development in deep learning provide a viable solution f

In [16]:
chunks[11].metadata.orig_elements

In [17]:
chunks[11].metadata.orig_elements[9].to_dict()

{'type': 'Image',
 'element_id': 'cdcd977765a92352f7460704d54820d3',
 'text': 'Confusion Matrix 8 400 8 8 2 300 2 o oe o 2 8 o 2 200 = 3 a & -100 ° E c g -0 cocci healthy ned salmo Predicted labels',
 'metadata': {'coordinates': {'points': ((np.float64(149.4388888888889),
     np.float64(485.7825066666666)),
    (np.float64(149.4388888888889), np.float64(1122.9166666666665)),
    (np.float64(849.9981688888889), np.float64(1122.9166666666665)),
    (np.float64(849.9981688888889), np.float64(485.7825066666666))),
   'system': 'PixelSpace',
   'layout_width': 1700,
   'layout_height': 2200},
  'last_modified': '2025-10-03T08:34:34',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 6,
  'image_base64': '...',
  'image_mime_type': 'image/jpeg',
  'file_directory': '/content',
  'filename': '3723178.3723198.pdf'}}

In [18]:
from google.colab import userdata

In [20]:
def separate_content_types(chunk):
    """Analyze what types of content are in a chunk"""
    content_data = {
        'text': chunk.text,
        'tables': [],
        'images': [],
        'types': ['text']
    }

    # Check for tables and images in original elements
    if hasattr(chunk, 'metadata') and hasattr(chunk.metadata, 'orig_elements'):
        for element in chunk.metadata.orig_elements:
            element_type = type(element).__name__

            # Handle tables
            if element_type == 'Table':
                content_data['types'].append('table')
                table_html = getattr(element.metadata, 'text_as_html', element.text)
                content_data['tables'].append(table_html)

            # Handle images
            elif element_type == 'Image':
                if hasattr(element, 'metadata') and hasattr(element.metadata, 'image_base64'):
                    content_data['types'].append('image')
                    content_data['images'].append(element.metadata.image_base64)

    content_data['types'] = list(set(content_data['types']))
    return content_data

def create_ai_enhanced_summary(text: str, tables: List[str], images: List[str]) -> str:
    """Create AI-enhanced summary for mixed content"""

    try:
        # Initialize LLM (needs vision model for images)
        llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=userdata.get('OPENAI_API_KEY'))

        # Build the text prompt
        prompt_text = f"""You are creating a searchable description for document content retrieval.

        CONTENT TO ANALYZE:
        TEXT CONTENT:
        {text}

        """

        # Add tables if present
        if tables:
            prompt_text += "TABLES:\n"
            for i, table in enumerate(tables):
                prompt_text += f"Table {i+1}:\n{table}\n\n"

                prompt_text += """
                YOUR TASK:
                Generate a comprehensive, searchable description that covers:

                1. Key facts, numbers, and data points from text and tables
                2. Main topics and concepts discussed
                3. Questions this content could answer
                4. Visual content analysis (charts, diagrams, patterns in images)
                5. Alternative search terms users might use

                Make it detailed and searchable - prioritize findability over brevity.

                SEARCHABLE DESCRIPTION:"""

        # Build message content starting with text
        message_content = [{"type": "text", "text": prompt_text}]

        # Add images to the message
        for image_base64 in images:
            message_content.append({
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}
            })

        # Send to AI and get response
        message = HumanMessage(content=message_content)
        response = llm.invoke([message])

        return response.content

    except Exception as e:
        print(f"     ❌ AI summary failed: {e}")
        # Fallback to simple summary
        summary = f"{text[:300]}..."
        if tables:
            summary += f" [Contains {len(tables)} table(s)]"
        if images:
            summary += f" [Contains {len(images)} image(s)]"
        return summary

def summarise_chunks(chunks):
    """Process all chunks with AI Summaries"""
    print("🧠 Processing chunks with AI Summaries...")

    langchain_documents = []
    total_chunks = len(chunks)

    for i, chunk in enumerate(chunks):
        current_chunk = i + 1
        print(f"   Processing chunk {current_chunk}/{total_chunks}")

        # Analyze chunk content
        content_data = separate_content_types(chunk)

        # Debug prints
        print(f"     Types found: {content_data['types']}")
        print(f"     Tables: {len(content_data['tables'])}, Images: {len(content_data['images'])}")

        # Create AI-enhanced summary if chunk has tables/images
        if content_data['tables'] or content_data['images']:
            print(f"     → Creating AI summary for mixed content...")
            try:
                enhanced_content = create_ai_enhanced_summary(
                    content_data['text'],
                    content_data['tables'],
                    content_data['images']
                )
                print(f"     → AI summary created successfully")
                print(f"     → Enhanced content preview: {enhanced_content[:200]}...")
            except Exception as e:
                print(f"     ❌ AI summary failed: {e}")
                enhanced_content = content_data['text']
        else:
            print(f"     → Using raw text (no tables/images)")
            enhanced_content = content_data['text']

        # Create LangChain Document with rich metadata
        doc = Document(
            page_content=enhanced_content,
            metadata={
                "original_content": json.dumps({
                    "raw_text": content_data['text'],
                    "tables_html": content_data['tables'],
                    "images_base64": content_data['images']
                })
            }
        )

        langchain_documents.append(doc)

    print(f"✅ Processed {len(langchain_documents)} chunks")
    return langchain_documents


# Process chunks with AI
processed_chunks = summarise_chunks(chunks)

🧠 Processing chunks with AI Summaries...
   Processing chunk 1/21
     Types found: ['text', 'image']
     Tables: 0, Images: 1
     → Creating AI summary for mixed content...
     → AI summary created successfully
     → Enhanced content preview: **Document Title:** An Ensemble Deep Learning Approach to Detect Common Chicken Diseases from Fecal Matter Images

**Authors and Affiliations:**
- Al-Momen Reyad, Computer Science and Engineering, Uni...
   Processing chunk 2/21
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 3/21
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 4/21
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 5/21
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 6/21
     Types found: ['text']
     Tables: 0, Imag

In [21]:
processed_chunks

[Document(metadata={'original_content': '{"raw_text": "Cheek for updates.\\n\\nAn Ensemble Deep Learning Approach to Detect Common Chicken Diseases from Fecal Matter Images\\n\\nAl-Momen Reyad Computer Science and Engineering United International University Dhaka, Bangladesh areyad203011@bscse.uiu.ac.bd\\n\\nSonjoy Dey\\n\\nComputer Science and Engineering United International University Dhaka, Bangladesh sdey202074@bscse.uiu.ac.bd\\n\\nWasi Uddin Bhuyian Computer Science and Engineering United International University Dhaka, Bangladesh wbhuyian201286@bscse.uiu.ac.bd\\n\\nSahadat Islam Evan\\n\\nComputer Science and Engineering United International University Dhaka, Bangladesh sevan203030@bscse.uiu.ac.bd", "tables_html": [], "images_base64": ["/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCABIAEgDASIAAhEBAxEB/8QAHwAAAQ

In [22]:
def export_chunks_to_json(chunks, filename="chunks_export.json"):
    """Export processed chunks to clean JSON format"""
    export_data = []

    for i, doc in enumerate(chunks):
        chunk_data = {
            "chunk_id": i + 1,
            "enhanced_content": doc.page_content,
            "metadata": {
                "original_content": json.loads(doc.metadata.get("original_content", "{}"))
            }
        }
        export_data.append(chunk_data)

    # Save to file
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, indent=2, ensure_ascii=False)

    print(f"✅ Exported {len(export_data)} chunks to {filename}")
    return export_data

# Export your chunks
json_data = export_chunks_to_json(processed_chunks)

✅ Exported 21 chunks to chunks_export.json


In [24]:
def create_vector_store(documents, persist_directory="db_v1/chroma_db"):
  print("Creating embeddings and storing in ChromaDB...")
  embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", api_key=userdata.get('OPENAI_API_KEY'))

  print("-----Creating Vector Store-------")
  vectorstore = Chroma.from_documents(
      documents=documents,
      embedding=embedding_model,
      persist_directory=persist_directory,
      collection_metadata={"hnsw:space":"cosine"}
  )

  print("--------Finished creating vector store------")

  return vectorstore

db=create_vector_store(processed_chunks)

Creating embeddings and storing in ChromaDB...
-----Creating Vector Store-------
--------Finished creating vector store------


In [25]:
query = "What is the percentage of healthy class in the dataset?"
retriever = db.as_retriever(search_kwargs={"k":3})
chunks=retriever.invoke(query)

export_chunks_to_json(chunks, "rag_retireve_result.json")

✅ Exported 3 chunks to rag_retireve_result.json


[{'chunk_id': 1,
  'enhanced_content': '3 METHODOLOGY\n\nThe objective is to identify diseases in chickens from image analysis of their feces. The model will help to understand if the chicken is minimal if healthy and otherwise, discover the disease.\n\n3.1 Experimental Design\n\nThe detailed stream of the experimental design, illustrated in Fig 1, starts with the appropriate preprocessing of the dataset, using state- of-art data augmentation techniques and mentation strategies that led to increased diversity representation and improved effectiveness quality. Fig 2 shows the complete data set and is followed by That is, the data is divided into separate groups by different characteristics.\n\nThe dataset used in the study is split equally into 80% data for training and the remaining for testing, to provide the basis for an effective evaluation of the obtained settings ensuring, at the same time, the sufficient comparison of model generalization. For model training, three CNN-based arch

In [26]:
def run_complete_ingestion_pipeline(pdf_path: str):
    """Run the complete RAG ingestion pipeline"""
    print("Starting RAG Ingestion Pipeline")
    print("=" * 50)

    # Step 1: Partition
    elements = partition_document(pdf_path)

    # Step 2: Chunk
    chunks = create_chunks_by_title(elements)

    # Step 3: AI Summarisation
    summarised_chunks = summarise_chunks(chunks)

    # Step 4: Vector Store
    db = create_vector_store(summarised_chunks, persist_directory="db_v2/chroma_db")

    print("Pipeline completed successfully!")
    return db

In [27]:
db = run_complete_ingestion_pipeline("/content/3723178.3723198.pdf")


Starting RAG Ingestion Pipeline
partition document: /content/3723178.3723198.pdf
Extracted 195 elements
Created 21 chunks
🧠 Processing chunks with AI Summaries...
   Processing chunk 1/21
     Types found: ['text', 'image']
     Tables: 0, Images: 1
     → Creating AI summary for mixed content...
     → AI summary created successfully
     → Enhanced content preview: **Document Title:** An Ensemble Deep Learning Approach to Detect Common Chicken Diseases from Fecal Matter Images

**Authors and Affiliations:**
- Al-Momen Reyad, Sonjoy Dey, Wasi Uddin Bhuyian, Sahad...
   Processing chunk 2/21
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 3/21
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 4/21
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 5/21
     Types found: ['text']
     Tables: 0, I

In [29]:
query = "What is the percentage of healthy class in the dataset?"

retriever = db.as_retriever(search_kwargs={"k": 3})
chunks = retriever.invoke(query)

def generate_final_answer(chunks, query):
    """Generate final answer using multimodal content"""

    try:
        # Initialize LLM (needs vision model for images)
        llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=userdata.get('OPENAI_API_KEY'))

        # Build the text prompt
        prompt_text = f"""Based on the following documents, please answer this question: {query}

CONTENT TO ANALYZE:
"""

        for i, chunk in enumerate(chunks):
            prompt_text += f"--- Document {i+1} ---\n"

            if "original_content" in chunk.metadata:
                original_data = json.loads(chunk.metadata["original_content"])

                # Add raw text
                raw_text = original_data.get("raw_text", "")
                if raw_text:
                    prompt_text += f"TEXT:\n{raw_text}\n\n"

                # Add tables as HTML
                tables_html = original_data.get("tables_html", [])
                if tables_html:
                    prompt_text += "TABLES:\n"
                    for j, table in enumerate(tables_html):
                        prompt_text += f"Table {j+1}:\n{table}\n\n"

            prompt_text += "\n"

        prompt_text += """
Please provide a clear, comprehensive answer using the text, tables, and images above. If the documents don't contain sufficient information to answer the question, say "I don't have enough information to answer that question based on the provided documents."

ANSWER:"""

        # Build message content starting with text
        message_content = [{"type": "text", "text": prompt_text}]

        # Add all images from all chunks
        for chunk in chunks:
            if "original_content" in chunk.metadata:
                original_data = json.loads(chunk.metadata["original_content"])
                images_base64 = original_data.get("images_base64", [])

                for image_base64 in images_base64:
                    message_content.append({
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}
                    })

        # Send to AI and get response
        message = HumanMessage(content=message_content)
        response = llm.invoke([message])

        return response.content

    except Exception as e:
        print(f"❌ Answer generation failed: {e}")
        return "Sorry, I encountered an error while generating the answer."

# Usage
final_answer = generate_final_answer(chunks, query)
print(final_answer)

The percentage of the healthy class in the dataset is 26%.
